In [1]:
import requests
import pandas as pd
import glob

In [3]:
# récupérer tous les fichiers du dossier videos et dédupliquer les lignes
files = glob.glob("market_divertissement_fr.csv")

df = []
for f in files:
    csv = pd.read_csv(f)
    df.append(csv)
df = pd.concat(df,ignore_index=True)
df.drop_duplicates(subset=['video_id'], keep="first", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   video_id      1000 non-null   object
 1   title         1000 non-null   object
 2   description   586 non-null    object
 3   channel_id    1000 non-null   object
 4   category_id   1000 non-null   int64 
 5   language      985 non-null    object
 6   channel       1000 non-null   object
 7   published_at  1000 non-null   object
 8   duration      1000 non-null   object
 9   views         1000 non-null   int64 
 10  likes         1000 non-null   int64 
 11  comments      1000 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 93.9+ KB


In [5]:
df.head()

,video_id,title,description,channel_id,category_id,language,channel,published_at,duration,views,likes,comments
0,-khN-1sBess,Vous pensez qu’il a perdu quoi ? 😂 #divertisse...,NaN,UCQ-EzEQR_4nJfjASQEvDz_A,22,fr,paul roja,2026-01-20T11:48:25Z,PT13S,21512,107,7
1,rH7wT7TwiNM,Le ninja le plus dangereux,NaN,UC5fZptHCu8qpCWBQ-6WiGdw,22,fr,V Film,2025-03-25T14:01:00Z,PT53S,4560728,102854,238
2,NMA9uHhx4sE,Baby Baby Baby Challenge,NaN,UClPGrZvHId38DxlURApQu8A,24,en-US,Deni Medeiros,2026-01-12T13:30:26Z,PT1M17S,8006373,119418,226
3,oDaAtVvEHPc,WENN ICH DU WÄRE… (extrem lustig 😂!!) #turnen ...,NaN,UCWavuOpKwQH72kaEAkDNqyA,24,de,Merle Loosen,2025-09-28T11:01:33Z,PT42S,5833145,64432,197
4,mKWrxAMokyU,Viral Shorts🫶🏻|GAME ZONE| #viralshorts #challe...,NaN,UCTzTT35pjMGxaK7w57SiCaA,23,hi,Time Waste,2026-01-22T06:16:01Z,PT57S,5893,101,2


In [7]:
# langues: supprimer les lignes avec valeurs
df.dropna(subset="language",axis=0,inplace=True)

# langues: regrouper fr_fr, be_fr sous 'fr', pareil pour 'en'
df.loc[df['language'].str.contains('^fr'),'language'] = "fr"
df.loc[df['language'].str.contains('^en'),'language'] = "en"

In [9]:
#supprimer autres langues
df['language'].value_counts()

language
fr         498
en         216
hi         166
und         12
ta          11
de          10
id           9
ar           9
zxx          8
ur           7
es           7
or           5
bn           4
bn-IN        3
pl           3
de-DE        2
tr           2
te           2
it           2
uz           2
ru           1
zh-CN        1
zh           1
ne           1
ml           1
hi-Latn      1
cs           1
Name: count, dtype: int64

In [11]:
# calculer le taux d'engagement pour voir en un coup d'oeil les vidéos ayant reçu le plus d'intéractions
df["Engagement total"] = df["likes"] + df["comments"]
df["Taux d'engagement (%)"] = (df["Engagement total"] / df["views"].replace(0, 1)) * 100

In [13]:
df.head()

,video_id,title,description,channel_id,category_id,language,channel,published_at,duration,views,likes,comments,Engagement total,Taux d'engagement (%)
0,-khN-1sBess,Vous pensez qu’il a perdu quoi ? 😂 #divertisse...,NaN,UCQ-EzEQR_4nJfjASQEvDz_A,22,fr,paul roja,2026-01-20T11:48:25Z,PT13S,21512,107,7,114,0.529937
1,rH7wT7TwiNM,Le ninja le plus dangereux,NaN,UC5fZptHCu8qpCWBQ-6WiGdw,22,fr,V Film,2025-03-25T14:01:00Z,PT53S,4560728,102854,238,103092,2.260429
2,NMA9uHhx4sE,Baby Baby Baby Challenge,NaN,UClPGrZvHId38DxlURApQu8A,24,en,Deni Medeiros,2026-01-12T13:30:26Z,PT1M17S,8006373,119418,226,119644,1.494360
3,oDaAtVvEHPc,WENN ICH DU WÄRE… (extrem lustig 😂!!) #turnen ...,NaN,UCWavuOpKwQH72kaEAkDNqyA,24,de,Merle Loosen,2025-09-28T11:01:33Z,PT42S,5833145,64432,197,64629,1.107961
4,mKWrxAMokyU,Viral Shorts🫶🏻|GAME ZONE| #viralshorts #challe...,NaN,UCTzTT35pjMGxaK7w57SiCaA,23,hi,Time Waste,2026-01-22T06:16:01Z,PT57S,5893,101,2,103,1.747836


In [15]:
# transofrmer les durées (ici min/sec) en secondes (plus simple pour l'analyse
import re

# 1. Conversion en timedelta (errors='coerce' transforme les formats invalides en NaT)
df['duration_td'] = pd.to_timedelta(df['duration'], errors='coerce')

# 2. Conversion en secondes
# .astype('Int64') permet de garder les NaN si nécessaire sans faire planter le code
df['Durée (s)'] = df['duration_td'].dt.total_seconds().astype('Int64')

# OPTIONNEL : Si vous voulez remplacer les vides par 0 et avoir des entiers classiques
# df['Durée (s)'] = df['duration_td'].dt.total_seconds().fillna(0).astype(int)

print(df[['duration', 'Durée (s)']])

    duration  Durée (s)
0      PT13S         13
1      PT53S         53
2    PT1M17S         77
3      PT42S         42
4      PT57S         57
..       ...        ...
995   PT1M3S         63
996  PT2M39S        159
997    PT15S         15
998    PT59S         59
999    PT53S         53

[985 rows x 2 columns]


In [17]:
df.head()

,video_id,title,description,channel_id,category_id,language,channel,published_at,duration,views,likes,comments,Engagement total,Taux d'engagement (%),duration_td,Durée (s)
0,-khN-1sBess,Vous pensez qu’il a perdu quoi ? 😂 #divertisse...,NaN,UCQ-EzEQR_4nJfjASQEvDz_A,22,fr,paul roja,2026-01-20T11:48:25Z,PT13S,21512,107,7,114,0.529937,0 days 00:00:13,13
1,rH7wT7TwiNM,Le ninja le plus dangereux,NaN,UC5fZptHCu8qpCWBQ-6WiGdw,22,fr,V Film,2025-03-25T14:01:00Z,PT53S,4560728,102854,238,103092,2.260429,0 days 00:00:53,53
2,NMA9uHhx4sE,Baby Baby Baby Challenge,NaN,UClPGrZvHId38DxlURApQu8A,24,en,Deni Medeiros,2026-01-12T13:30:26Z,PT1M17S,8006373,119418,226,119644,1.494360,0 days 00:01:17,77
3,oDaAtVvEHPc,WENN ICH DU WÄRE… (extrem lustig 😂!!) #turnen ...,NaN,UCWavuOpKwQH72kaEAkDNqyA,24,de,Merle Loosen,2025-09-28T11:01:33Z,PT42S,5833145,64432,197,64629,1.107961,0 days 00:00:42,42
4,mKWrxAMokyU,Viral Shorts🫶🏻|GAME ZONE| #viralshorts #challe...,NaN,UCTzTT35pjMGxaK7w57SiCaA,23,hi,Time Waste,2026-01-22T06:16:01Z,PT57S,5893,101,2,103,1.747836,0 days 00:00:57,57
